In [3]:
# TumourScan-AI: Brain Tumour Classifier using PyTorch
# 1. Data Preprocessing

# Import PyTorch and supporting libraries
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Detect if GPU (CUDA) is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define image transformations:
tf = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load training data from the 'data/Training' folder
train_dl = DataLoader(
    datasets.ImageFolder('data/Training', tf),
    batch_size = 32, shuffle = True, num_workers = 4, pin_memory = True
)

# Load testing data from the 'data/Testing' folder
test_dl = DataLoader(
    datasets.ImageFolder('data/Testing', tf),
    batch_size = 32, shuffle = False, num_workers = 4, pin_memory = True
)

In [4]:
# 2. Define the CNN Model
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128 * 16 * 16, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 4)
).to(device)

In [5]:
# 3. Set up Optimizer and Loss Function
opt = optim.AdamW(model.parameters(), 1e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# 4. Training Loop
model.train()

# Train for 25 epochs
for epoch in range(25):
    running_loss = 0

    # Loop through training batches
    for x, y in train_dl:
        opt.zero_grad()
        
        # Forward pass: get model predictions
        loss = loss_fn(model(x.to(device)), y.to(device))
        loss.backward()

        running_loss += loss

        opt.step()

    print(f'Epoch {epoch + 1}: Loss was {running_loss}') 

In [ ]:
# 5. Model Evaluation
model.eval()
test_loss, correct = 0.0, 0

# Disable gradient calculation for faster testing
with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(device), y.to(device)

        # Forward pass for predictions
        logits = model(x)
        # Compute loss for the batch
        test_loss += loss_fn(logits, y).item() * y.size(0)

        # Get predicted class for each sample
        preds = logits.argmax(dim = 1)
        # Count how many predictions were correct
        correct += (preds == y).sum().item()

# Compute average loss and overall accuracy
test_loss /= len(test_dl.dataset)
accuracy = 100.0 * correct / len(test_dl.dataset)

print('Test Loss:', test_loss, 'Test accuracy', accuracy, "%")

In [ ]:
# 6. Visualize a Random Test Image
import random

import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image

model.eval()

# Pick a random test sample
idx = random.randrange(len(test_dl.dataset))
# Get image and label
img, label = test_dl.dataset[idx]

# Undo normalization for visualization
unnorm = img * 0.5 + 0.5
plt.imshow(to_pil_image(unnorm))
plt.axis('off')
plt.title('Sample from test set')
plt.show()

# Predict the tumour class for this image
with torch.no_grad():
    # Add batch dimension
    logits = model(img.unsqueeze(0).to(device))
    # Get predicted label index
    pred = logits.argmax(1).item()

# Retrieve class names
class_names = test_dl.dataset.classes
print(f"Predicted class: {class_names[pred]}")
print(f"Ground-truth: {class_names[label]}")